---

## 
- **Usage**: Optimized for automated workloads in production
- **Management**: ephemeral, automatically provisioned by the Databricks job scheduler when job is triggered
- **Termination**: Designed for a single purpose and terminate automatically and immediately after the job is completed
- **Cost efficiency**: cost less than all purpose and warehouse. $0.20 per DBU for classic and $0.45 for serveless
    https://www.databricks.com/product/pricing/lakeflow-jobs

---